# Lab 6: Solución de sistemas de ecuaciones lineales

In [1]:
import numpy as np
import scipy as sc
import scipy.linalg
import math
import matplotlib.pyplot as plt
import time
import pandas as pd

## 1)

Escribir `soltrsup` y `soltrinf` que resuelvan el sistema lineal $ Ax = b $, donde $ A $ es una matriz triangular (superior e inferior, respectivamente). La entrada debe ser $ (A, b) $ con $ A \in \mathbb{R}^{nxn} $ matriz triangular y $ b \in \mathbb{R} $, y la salida debe ser la solución $ x $. Se debe imprimir un mensaje de error si la matriz es singular.

In [2]:
def soltrinf(A, b):
    
    # comenzamos verificando que no sea singular
    # es decir, que su determinante es distinto de cero
    assert np.linalg.det(A) != 0, 'A no puede ser singular'
    assert A.shape[0] == b.shape[0], 'A y b deben tener la misma altura'
    
    # definimos N en base a la dimensión de la matriz A
    N = A.shape[0]
    
    # caculamos el x
    x = np.zeros_like(b, dtype=np.float)
    for i in range(N):
        # buscamos la suma de los productos
        # entre la sección triangular inferior
        # y lo que calculamos ya de X
        s = sum([A[i, j] * x[j] for j in range(i)])
        
        # buscamos el resultado final de x[i]
        x[i] = (b[i] - s) / A[i, i]
        
    return x


def soltsup(A, b):
    
    # comenzamos verificando que no sea singular
    # es decir, que su determinante es distinto de cero
    assert np.linalg.det(A) != 0, 'A no puede ser singular'
    assert A.shape[0] == b.shape[0], 'A y b deben tener la misma altura'
    
    # definimos N en base a la dimensión de la matriz A
    N = A.shape[0]
    
    # caculamos el x
    x = np.zeros_like(b, dtype=np.float)
    for i in reversed(range(N)):
        
        # buscamos la suma de los productos
        # entre la sección triangular superior
        # y lo que calculamos de X
        s = sum([A[i, j] * x[j] for j in range(i + 1, N)])
        
        # buscamos el resultado final de x[i]
        x[i] = (b[i] - s) / A[i, i]
        
    return x


Probamos las funciones con matrices conocidas ([https://matrixcalc.org/en/slu.html]()):

$$
\left(\begin{array}{cc}
1 & 0 & 0 \\
5 & 5 & 0 \\
3 & 2 & 1
\end{array}\right)
\left(\begin{array}{cc}
2 \\
-7/5 \\
-6/5
\end{array}\right)
=
\left(\begin{array}{cc}
2 \\
3 \\
2
\end{array}\right)
$$ 

In [3]:
A = np.stack([[1, 0, 0],
              [5, 5, 0],
              [3, 2, 1]])
b = np.array([2, 3, 2])

x = soltrinf(A, b)
x

array([ 2. , -1.4, -1.2])

$$
\left(\begin{array}{cc}
1 & 5 & 3 \\
0 & 5 & 5 \\
0 & 0 & 1
\end{array}\right)
\left(\begin{array}{cc}
3 \\
-7/5 \\
2
\end{array}\right)
=
\left(\begin{array}{cc}
2 \\
3 \\
2
\end{array}\right)
$$ 

In [4]:
A = np.stack([[1, 5, 3],
              [0, 5, 5],
              [0, 0, 1]])
b = np.array([2, 3, 2])

x = soltsup(A, b)
x

array([ 3. , -1.4,  2. ])

## 2) 

### a) 

Escribir una función llamada `egauss` que implemente el método de la eliminación Gaussiana. Este método nos ayuda a simplificar sistemas representados con matrices triangulares superiores (no necesariamente no singulares), de manera que puedan ser resueltos por el método descripto en el ejercicio anterior.

Por esta razón, la entrada de la función debería ser una matrix y un vector, y se devuelven $ U, y $, correspondientes a $ A, b $ transformados.

In [5]:
def egauss(A, b):
    
    # definimos N en base a la dimensión de A
    N = A.shape[0]
    
    # copiamos los datos de entrada
    # los cuales iremos modificando para simplificar
    # el sistema
    U, y = A.copy(), b.copy()
    U, y = U.astype(np.float), y.astype(np.float)
    
    for k in range(N - 1):
        for i in range(k + 1, N):            
            assert A[k, k] != 0, f'Diag Cero:  A{k}{k} = 0'
            
            # calculamos los nuevos valores
            # de nuestra matriz / vector
            m = U[i, k] / U[k, k]

            for j in range(k + 1, N):
                U[i, j] = U[i, j] - m * U[k, j]

            y[i] = y[i] - m*y[k]
    
    # convertimos a cero todos los valores
    # por debajo de la diagonal de U 
    # ya que queremos una matriz triangular superior no singular
    U = np.triu(U)

    return U, y


## b)

Escribir una función llamada `soleg` que resuelva sistemas lineales $ Ax = b $ utilizando eliminación Gaussiana y resolviendo el sistema triangular superior.

In [6]:

def soleg(A, b):
    
    # conseguimos la simplificación
    U, y = egauss(A, b)
    
    # resolvemos el sistema triangular superior
    x = soltsup(U, y)
    
    return x

Probamos una solución conocida:

$$
\left(\begin{array}{cc}
6 & -2 & 2 & 4 \\
12 & -8 & 6 & 10 \\
3 & -13 & 9 & 3 \\
-6 & 4 & 1 & -18
\end{array}\right)
\left(\begin{array}{cc}
1 \\
-3 \\
-2 \\
1
\end{array}\right)
=
\left(\begin{array}{cc}
12 \\
34 \\
27 \\
-38
\end{array}\right)
$$ 

In [7]:
# construir las matrices
A = np.stack([[6, -2, 2, 4],
              [12, -8, 6, 10],
              [3, -13, 9, 3],
              [-6, 4, 1, -18]])

b = np.array([12, 34, 27, -38])

# usamos soleg

x = soleg(A, b)

x

array([ 1., -3., -2.,  1.])

## 3)

Escribir una función llamada `sollu` que resuelva sistemas lineales $ Ax = b $ usando descomposición LU con pivoteo (para obtener dicha descomposición investigar el subpaquete de la librería scipy: `linalg`) para luego resolver $ Ly = P^-1 b $ (¿Cómo se puede obtener la inversa de una matriz de pivoteo?) y $ Ux = y $ usando `soltrinf` y `soltrsup`. la salida debe ser la solución $x$ y debe tener de entrada $ (A, b) $ con $ A \in \mathbb R^{nxn} $ y $ b \in \mathbb R^n $. 

Primero comenzamos investigando cómo se puede descomponer una matriz A:

In [8]:
A = np.random.randn(4, 4)
P, L, U = sc.linalg.lu(A)

A.shape, P.shape, L.shape, U.shape

((4, 4), (4, 4), (4, 4), (4, 4))

Ahora que tenemos estos datos, construimos nuestra función:

In [9]:
def sollu(A, b):
    # buscamos la descomposición
    P, L, U = sc.linalg.lu(A.astype(np.float))
    
    # resolvemos Ly = P^(-1)b
    # P.T es la inversa de P
    # @ es el operador para multiplicar ndarrays
    y = soltrinf(L, P.T @ b)
    
    # resolvemos Ux = y
    x = soltsup(U, y)
    
    return x

In [10]:
# construir las matrices
A = np.stack([[6, -2, 2, 4],
              [12, -8, 6, 10],
              [3, -13, 9, 3],
              [-6, 4, 1, -18]])

b = np.array([12, 34, 27, -38])

# usamos soleg

x = sollu(A, b)

x

array([ 1., -3., -2.,  1.])

## 4)

Comparar las soluciones dadas por `soleg` y `sollu` al resolver $ Ax = b $ con:

$$
A = 
\left(\begin{array}{cc}
4 & -1 & 0 & -1 & 0 & 0 \\
-1 & 4 & -1 & 0 & -1 & 0 \\
0 & -1 & 4 & 0 & 0 & -1 \\
-1 & 0 & 0 & 4 & -1 & 0 \\
0 & -1 & 0 & -1 & 4 & -1 \\
0 & 0 & -1 & 0 & -1 & 4 \\
\end{array}\right)
 , b1 = 
\left(\begin{array}{cc}
1 \\
1 \\
1 \\
0 \\
0 \\
0 \\
\end{array}\right)
, b2 =
\left(\begin{array}{cc}
1 \\
1 \\
1 \\
1 \\
1 \\
1 \\
\end{array}\right)
$$

Para comparar los dos métodos, no sólo vamos a usar los resultados, si no que vamos a calcular cuánto tiempo lleva cada uno.

In [11]:
# definimos los parámetros

A = np.matrix([
    [4, -1, 0, -1, 0, 0],
    [-1, 4, -1, 0, -1, 0],
    [0, -1, 4, 0, 0, -1],
    [-1, 0, 0, 4, -1, 0],
    [0, -1, 0, -1, 4, -1],
    [0, 0, -1, 0, -1, 4],
])

b_1 = np.array([1, 1, 1, 0, 0, 0])
b_2 = np.array([1, 1, 1, 1, 1, 1])

# método soleg con b1

soleg_b1_start = time.process_time()
soleg_x_1 = soleg(A, b_1)
soleg_b1_end = time.process_time()

# método soleg con b2

soleg_b2_start = time.process_time()
soleg_x_2 = soleg(A, b_2)
soleg_b2_end = time.process_time()

# método sollu con b1

sollu_b1_start = time.process_time()
sollu_x_1 = sollu(A, b_1)
sollu_b1_end = time.process_time()

# método sollu con b2

sollu_b2_start = time.process_time()
sollu_x_2 = sollu(A, b_2)
sollu_b2_end = time.process_time()

# validamos que los resultados
# sean lo suficientemente parecidos

assert np.isclose(soleg_x_1, sollu_x_1, 0.0001).all(), 'Para b1 son distintos'
assert np.isclose(soleg_x_2, sollu_x_2, 0.0001).all(), 'Para b2 son distintos'

# mostramos los resultados:

print(f'Método soleg con b1, tiempo = {soleg_b1_end - soleg_b1_start}')
print(f'Método sollu con b1, tiempo = {sollu_b1_end - sollu_b1_start}')

print(f'Método soleg con b2, tiempo = {soleg_b2_end - soleg_b2_start}')
print(f'Método sollu con b2, tiempo = {sollu_b2_end - sollu_b2_start}')

Método soleg con b1, tiempo = 0.0017739999999999423
Método sollu con b1, tiempo = 0.0011450000000001737
Método soleg con b2, tiempo = 0.002030999999999894
Método sollu con b2, tiempo = 0.0009600000000000719


Cómo podemos observar, en la mayoría de los casos, el método `soleg` es mucho más rápido que `sollu`.

## 5)

Escribir dos funciones llamadas `jacobi` y  `gseidel` que resuelvan sistemas lineales $ Ax = b $ usando los respectivos métodos. La salida debe ser $ [x, k] $ donde $ x $ es la solución aproximada y $ k $ la cantidad de iteraciones realizadas. Debe tener entrada $ (A, b, err, mit) $ con $ A \in \mathbb R^(nxn), b \in \mathbb R^n $, err es la tolerancia de error y mit es la cantidad máxima de iteraciones a realizar. El algoritmo debe parar si $ || x^{(k)} - x^{(k - 1)} ||_{\infty} \leq err $ o $ k \geq mit $.

In [12]:
def jacobi(A, b, err, mit):
    # convertimos todo a flotante
    A = A.astype(np.float)
    b = b.astype(np.float)
    
    # inicializamos x con ceros y definimos N
    x = np.zeros_like(b)
    N = A.shape[0]
    
    for k in range(mit):
        
        # buscamos una nueva aproximación
        u = x.copy()
        for i in range(N):
            sum_i = sum([A[i, j] * x[j] for j in range(N) if j != i])
            u[i] = (1 / A[i, i]) * (b[i] - sum_i)
        
        # validamos y pisamos x
        if (np.abs(u - x) < err).all():
            return u, k
        x = u
    
    return x, mit

In [13]:
# probamos jacobi

A = np.matrix([
    [7, -6],
    [-8, 9]
])
b = np.array([3, -4])

x, k = jacobi(A, b, 0.0001, 100)
x

array([ 0.20004988, -0.26670546])

In [14]:
def gseidel(A, b, err, mit):
    # convertimos todo a flotante
    A = A.astype(np.float)
    b = b.astype(np.float)
    
    # inicializamos x con ceros y definimos N
    x = np.zeros_like(b)
    N = A.shape[0]
    
    for k in range(mit):
        
        # buscamos una aproximación
        u = np.zeros_like(x)
        for i in range(N):
            sum_i = sum([A[i, j] * x[j] for j in range(N) if j != i])
            u[i] = (1 / A[i, i]) * (b[i] - sum_i)
        
        # validamos y actualizamos x
        if (np.abs(u - x) < err).all():
            return u, k
        
        x = u
    
    return x, mit

In [16]:
# probamos Gauss Seidel

A = np.matrix([
    [7, -6],
    [-8, 9]
])
b = np.array([3, -4])

x, k = gseidel(A, b, 0.0001, 100)
x

array([ 0.20004988, -0.26670546])

## 6)

Usar los métodos de Jacobi y Gauss-Seidel para resolver:

$$
(1) 
\left(\begin{array}{cc}
3 & 1 & 1 \\
2 & 6 & 1 \\
1 & 1 & 4 \\
\end{array}\right)
\left(\begin{array}{cc}
x_1 \\
x_2 \\
x_3 \\
\end{array}\right)
=
\left(\begin{array}{cc}
5 \\
9 \\
6 \\
\end{array}\right)
, (2) 
\left(\begin{array}{cc}
5 & 7 & 6 & 5 \\
7 & 10 & 8 & 7 \\
6 & 8 & 10 & 9 \\
5 & 7 & 9 & 10 \\
\end{array}\right)
\left(\begin{array}{cc}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
\end{array}\right)
=
\left(\begin{array}{cc}
23 \\
32 \\
33 \\
31 \\
\end{array}\right)
$$



In [18]:
# (1)

A = np.matrix([
    [3, 1, 1],
    [2, 6, 1],
    [1, 1, 4],
])
b = np.array([5, 9, 6])
err = pow(10, -11)

jx, jk = jacobi(A, b, err, 500)
gx, gk = gseidel(A, b, err, 500)

print(
f'Jacobi: x = {jx}, k = {jk} \n'
f'Gauss-Seidel: x = {gx}, k = {gk}'
)

Jacobi: x = [1. 1. 1.], k = 45 
Gauss-Seidel: x = [1. 1. 1.], k = 45


In [21]:
# (2)

A = np.matrix([
    [5, 7, 6, 5],
    [7, 10, 8, 7],
    [6, 8, 10, 9],
    [5, 7, 9, 10]
])
b = np.array([23, 32, 33, 31])
err = pow(10, -11)

jx, jk = jacobi(A, b, err, 500)
gx, gk = gseidel(A, b, err, 500)

print(
f'Jacobi: x = {jx}, k = {jk} \n'
f'Gauss-Seidel: x = {gx}, k = {gk}'
)

Jacobi: x = [-9.62494568e+196 -6.69787281e+196 -6.78902817e+196 -6.30550249e+196], k = 500 
Gauss-Seidel: x = [-9.62494568e+196 -6.69787281e+196 -6.78902817e+196 -6.30550249e+196], k = 500
